In [136]:
%matplotlib inline
from __future__ import division, print_function
import george 
from george import kernels
import numpy as np 

## testing the parametrization of the exponent squared kernel

The  kernel should give:

\begin{equation}
k(a^2, \tau; r_{ij}) = a^2  \exp(-r_{ij}^2 / 2\tau)
\end{equation}

\begin{equation}
C_{ij} = \sigma_n^2 \delta_{ij} + k(a^2, \tau; r_{ij})
\end{equation}

where $r_{ij}$ is the distance matrix

In [137]:
param = [0.1, 2]
sigma_n = 0.1 

In [138]:
gp = george.GP(param[0] * kernels.ExpSquaredKernel(param[1]))

## If we sample data at the following locations 

In [151]:
x_pos = np.arange(-2, 3)

In [152]:
x_pos

array([-2, -1,  0,  1,  2])

The expected covariance matrix $C$ should be: 

In [153]:
r = np.matrix(
    [[(x_pos[i] - x_pos[j]) * (x_pos[i] - x_pos[j])     
     for i in range(x_pos.size)]
     for j in range(x_pos.size)])
print(r)

[[ 0  1  4  9 16]
 [ 1  0  1  4  9]
 [ 4  1  0  1  4]
 [ 9  4  1  0  1]
 [16  9  4  1  0]]


In [165]:
C = param[0] * np.exp(- r / (2. * param[1]) ) + sigma_n ** 2  * np.eye(x_pos.size)

In [166]:
C

matrix([[ 0.11      ,  0.07788008,  0.03678794,  0.01053992,  0.00183156],
        [ 0.07788008,  0.11      ,  0.07788008,  0.03678794,  0.01053992],
        [ 0.03678794,  0.07788008,  0.11      ,  0.07788008,  0.03678794],
        [ 0.01053992,  0.03678794,  0.07788008,  0.11      ,  0.07788008],
        [ 0.00183156,  0.01053992,  0.03678794,  0.07788008,  0.11      ]])

this is the step to initiate the $\sigma_n$ term in the GP kernel

In [178]:
gp.compute(x_pos, yerr=sigma_n)
gp.get_matrix(x_pos)

array([[ 0.1       ,  0.07788008,  0.03678794,  0.01053992,  0.00183156],
       [ 0.07788008,  0.1       ,  0.07788008,  0.03678794,  0.01053992],
       [ 0.03678794,  0.07788008,  0.1       ,  0.07788008,  0.03678794],
       [ 0.01053992,  0.03678794,  0.07788008,  0.1       ,  0.07788008],
       [ 0.00183156,  0.01053992,  0.03678794,  0.07788008,  0.1       ]])

the diagonal elements do not match.
This is due to George only adding the diagonal terms at the C++ level code. See this [code line 66](https://github.com/dfm/george/blob/master/george/basic.py#L66)